In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from pprint import pprint
from time import sleep
import pandas as pd

In [2]:
def personal_info(link):

    browser.visit(link)
    per_info_soupy = soup(browser.html, "html.parser")
    athlete = per_info_soupy.find_all("div", class_="athlete")
    country = athlete[0].find("div", class_="country").find("span").text
    name = athlete[0].find("h1", class_="name").text
    age = athlete[0].find("span", class_="age").text
    hometown = athlete[0].find("span", class_="hometown").text
    info = per_info_soupy.find("div", class_="personal-info")
    athlete_info = info.find("div",class_="text")
    catergories = []
    data = []
    Athlete_infos = []
    try:   
        for i in athlete_info:
            if i["class"][0]=="subtitle":
                catergories.append(i.text)
            else:
                data.append(i.text)

        athlete_infos = {catergories[i]: data[i] for i in range(len(catergories))}
        Athlete_infos.append(athlete_infos)
        
    except:
        "NO PERSONAL INFORMATION"
    INFO =[country,name,age, Athlete_infos]
    return INFO
    

In [3]:
def climbers_stats(link):
    browser.visit(link)
    soupy = soup(browser.html, "html.parser")
    iframe = soupy.find('iframe')
    iframe_url = iframe['data-src']
    iframe_url
    browser.visit(iframe_url)
    return

In [4]:
def loop_table():
    iframesoup = soup(browser.html, "html.parser")
    table = iframesoup.find('table')
    climbers =[]
    for row in table.find_all('tr'):
        try:
            link = row.find_all('td')[1].find("a")["href"]
            climber = personal_info(link)
            climbers.append(climber)

        except:
            continue
    return climbers

In [5]:
def climbers(url):
    climbers_stats(url)
    sleep(1)
    climbers = loop_table()
    return climbers

In [6]:
def create_climber_df(url):
    competitors = climbers(url)
    df = pd.DataFrame(competitors)
    df = clean_climber_df(df)
    return df

In [7]:
def clean_climber_df(df):
    personal_info_list = pd.json_normalize(df[3])
    personl_info_df = pd.json_normalize(personal_info_list[0])
    df[1]=df[1].str.strip()
    name_df = df[1].str.split(" ",expand=True)
    for i in range(len(name_df.columns)):
        name_df.rename(columns={i:f"name_{i}"}, inplace=True)
    df[2]=df[2].str.strip("Age: ")
    df = df.merge(name_df, left_index=True, right_index=True)
    df = df.drop([1,3],axis=1)
    df = df.merge(personl_info_df,left_index=True, right_index=True)
    df = df.rename(columns={0:"country",2:"age"})
    return df

In [8]:
def women():
    women_url = 'https://www.ifsc-climbing.org/index.php/world-competition/calendar?task=ranking-complete&category=7'
    df = create_climber_df(women_url)
    return df

In [9]:
def men():
    men_url = 'https://www.ifsc-climbing.org/index.php/world-competition/calendar?task=ranking-complete&category=3'
    df = create_climber_df(men_url)
    return df

In [12]:
def scrape():
    women_df = women()
    men_df = men()
    browser.quit()
    
    women_df.to_csv('women_competitors.csv', index=False)
    men_df.to_csv('men_competitors.csv', index=False)

In [13]:
 browser = Browser('chrome')

In [14]:
scrape()